In [1]:
import os
import torch
from torch import nn,optim
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import io
from torchvision import datasets
from torchvision import transforms
from torchvision import models
import torchmetrics
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

# Data Preparation and Loading

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = io.read_image(img_path)/255
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [3]:
train_dataset = CustomImageDataset("data/FashionMNIST/train/images/",
                                   "data/FashionMNIST/train/labels.csv")
                                
test_dataset = CustomImageDataset("data/FashionMNIST/test/images/",
                                  "data/FashionMNIST/test/labels.csv")

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
len(train_dataset),len(test_dataset)
# test_dataset[0][1]

(59999, 9999)

# Build the model

In [4]:
class SimpleVisionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=784, out_features=10), # in_features = number of features in a data sample (784 pixels)
            nn.Linear(in_features=10, out_features=10),
        )
    def forward(self,X):
        return self.classifier(self.flatten(X))

model = SimpleVisionModel()
model(torch.randn((64,1,28,28)))

tensor([[ 6.1119e-01,  3.3034e-01,  4.2629e-01,  5.6226e-01, -3.9210e-01,
         -1.0968e-01,  3.2677e-01, -3.8377e-01,  5.8697e-01,  4.2494e-01],
        [ 7.7423e-01,  1.0280e-01, -1.7086e-01,  7.4224e-01, -3.2924e-01,
         -6.0679e-01,  8.4842e-02,  2.9762e-01, -2.4459e-01,  3.6022e-01],
        [-3.7514e-01,  3.0846e-01, -1.2450e-01, -3.5960e-01, -4.6238e-01,
         -7.0972e-01,  3.3950e-01,  3.5772e-02,  1.7466e-01,  8.2516e-02],
        [-6.3988e-01,  1.5957e-01,  1.0200e-03, -7.5042e-02, -4.2105e-01,
         -1.0169e-01,  5.2685e-01, -9.7499e-01,  3.3933e-01,  1.4153e-01],
        [ 5.1027e-01,  5.6917e-01, -1.9771e-01,  1.9321e-01,  8.6391e-02,
          3.4502e-01, -5.4847e-01, -6.7587e-01, -1.9903e-01,  1.5493e-01],
        [ 1.0977e-01,  1.1207e-01,  2.8547e-01,  7.9483e-01,  6.7805e-01,
         -3.0337e-01,  6.1187e-01,  5.2059e-01,  2.3048e-01,  3.4811e-01],
        [-1.4491e-01, -3.7375e-03,  1.0268e-01, -3.9782e-01, -5.7825e-01,
         -7.1492e-02, -2.8627e-0

# Train the model

In [9]:
optimizer = optim.SGD(model.parameters(),lr=0.1)
loss_fn = nn.CrossEntropyLoss()
accuracy_fn = torchmetrics.Accuracy(task="multiclass", num_classes=10)
EPOCHS = 3

torch.manual_seed(42)


epochs_counts = []
training_losses = []
testing_losses = []
training_accuracies = []
testing_accuracies = []
for epoch in tqdm.tqdm(range(EPOCHS)):
    print(f"Epoch: {epoch}\n-------")
    epochs_counts.append(epoch)
    # train
    train_loss,train_acc = 0,0
    for batch, (X, y) in enumerate(train_dataloader):
        model.train()
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_pred.argmax(dim=1).squeeze(),y) * 100
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    training_losses.append(train_loss)
    training_accuracies.append(train_acc)


    # test
    test_loss, test_acc = 0, 0 
    model.eval()
    with torch.inference_mode():
        for (X,y) in test_dataloader:
            y_test_pred = model(X)
            test_loss += loss_fn(y_test_pred,y).item()
            test_acc += accuracy_fn(y_test_pred.argmax(dim=1).squeeze(),y).item() * 100
    
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    testing_losses.append(test_loss)
    testing_accuracies.append(test_acc * 100)

    print(f"\nTrain loss: {train_loss} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/59999 samples
Looked at 25600/59999 samples
Looked at 51200/59999 samples


 33%|███▎      | 1/3 [00:13<00:27, 13.62s/it]


Train loss: 0.42361384256879914 | Test loss: 0.43144, Test acc: 85.66%

Epoch: 1
-------
Looked at 0/59999 samples
Looked at 25600/59999 samples
Looked at 51200/59999 samples


 67%|██████▋   | 2/3 [00:28<00:14, 14.36s/it]


Train loss: 0.4174228355542683 | Test loss: 0.44553, Test acc: 84.96%

Epoch: 2
-------
Looked at 0/59999 samples
Looked at 25600/59999 samples
Looked at 51200/59999 samples


100%|██████████| 3/3 [00:43<00:00, 14.38s/it]


Train loss: 0.4167349765867567 | Test loss: 0.43916, Test acc: 85.20%



In [ ]:
plt.plot(epochs_counts,training_losses)
plt.plot(epochs_counts,testing_losses)